In [ ]:
# Change the current directory to /content
%cd /content

# Clone the ComfyUI repository from GitHub
!git clone https://github.com/comfyanonymous/ComfyUI.git ./ComfyUI
!git clone https://github.com/city96/ComfyUI-GGUF.git ./ComfyUI/custom_nodes/quantized

In [ ]:
# Change the current directory to /content/ComfyUI
%cd /content/ComfyUI

# Install library
!pip install -q einops==0.8.0 torchsde==0.2.6 diffusers==0.31.0 accelerate==1.1.0 gguf==0.10.0 gradio==5.8.0
!pip install -q huggingface_hub==0.27.0 hf_transfer==0.1.8

In [ ]:
# Download pre-trained models using huggingface-cli

# Enable faster downloads with HF_HUB_ENABLE_HF_TRANSFER
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download YarvixPA/FLUX.1-Fill-dev-gguf flux1-fill-dev-Q4_K_S.gguf --local-dir ./models/unet --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download camenduru/FLUX.1-dev ae.sft --local-dir ./models/vae --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download zer0int/CLIP-GmP-ViT-L-14 ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors --local-dir ./models/clip --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp8_e4m3fn.safetensors --local-dir ./models/clip --local-dir-use-symlinks False

In [ ]:
# Change the current directory to /content/ComfyUI
%cd /content/ComfyUI

import os
import random
import torch
import numpy as np
from PIL import Image, ImageOps
import gradio as gr

# ComfyUI imports
import nodes
from nodes import NODE_CLASS_MAPPINGS
from comfy import model_management
from comfy_extras import (
    nodes_custom_sampler,
    nodes_differential_diffusion,
    nodes_flux,
    nodes_model_advanced
)

# Custom Node imports
from custom_nodes.quantized.nodes import NODE_CLASS_MAPPINGS as Q_NODE_CLASS_MAPPINGS

# Global references for nodes/models
CLIPVisionLoader = None
LoadImage = None
InpaintModelConditioning = None
DifferentialDiffusion = None
DualCLIPLoader = None
VAELoader = None
CLIPTextEncode = None
FluxGuidance = None
BasicGuider = None
KSampler = None
BasicScheduler = None
SamplerCustomAdvanced = None
RandomNoise = None
EmptyLatentImage = None
VAEDecode = None

clip = None
vae = None
unet = None

In [ ]:
def load_models():
    """
    Initializes and loads all required ComfyUI nodes and models.
    Sets global references to nodes and loaded models.
    """
    global CLIPVisionLoader, LoadImage, InpaintModelConditioning
    global DifferentialDiffusion, DualCLIPLoader, VAELoader
    global CLIPTextEncode, FluxGuidance, BasicGuider, KSampler
    global BasicScheduler, SamplerCustomAdvanced, RandomNoise
    global EmptyLatentImage, VAEDecode
    global clip, vae, unet

    # Instantiate node classes
    CLIPVisionLoader = NODE_CLASS_MAPPINGS["CLIPVisionLoader"]()
    LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
    InpaintModelConditioning = NODE_CLASS_MAPPINGS["InpaintModelConditioning"]()
    DifferentialDiffusion = nodes_differential_diffusion.NODE_CLASS_MAPPINGS["DifferentialDiffusion"]()

    DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
    VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
    CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
    FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
    BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
    KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
    BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
    SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()

    RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
    EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
    VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()

    # Load models
    with torch.inference_mode():
        # Load CLIP and VAE
        clip_model = DualCLIPLoader.load_clip(
            "t5xxl_fp8_e4m3fn.safetensors",
            "ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors",
            "flux"
        )[0]
        vae_model = VAELoader.load_vae("ae.sft")[0]

        # Load UNet
        UNETLoader = Q_NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
        unet_model = UNETLoader.load_unet("flux1-fill-dev-Q4_K_S.gguf")[0]

    # Assign globally
    global clip, vae, unet
    clip = clip_model
    vae = vae_model
    unet = unet_model


def closestNumber(n, m):
    """
    Finds the closest multiple of m to the number n.
    """
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2


def resize_max(image):
    """
    Resizes the image so that its longest side does not exceed 1024 pixels.
    """
    max_length = 1024
    width, height = image.size
    scaling_factor = min(max_length / width, max_length / height)

    if scaling_factor < 1:
        new_width = int(width * scaling_factor)
        new_height = int(height * scaling_factor)
        image = image.resize((new_width, new_height), Image.LANCZOS)

    return image


def generate_image(image, prompt, seed):
    """
    Generates an inpainted image using the loaded ComfyUI nodes and models.
    Takes an RGBA input image with mask information, a text prompt, and a seed.
    """
    with torch.inference_mode():
        steps = 20
        sampler_name = "euler"
        scheduler = "simple"

        image_path = image["background"]
        mask_path = image["layers"][0]

        # Convert input image and mask
        original_image = Image.open(image_path).convert("RGBA")
        original_image.save("/content/ComfyUI/input/original.png")
        mask = Image.open(mask_path).convert("L")
        mask = ImageOps.invert(mask)  # Invert mask for inpainting
        mask.save("/content/ComfyUI/input/mask.png")

        # Resize if needed
        width, height = original_image.size
        image_f = resize_max(original_image)
        mask_f = resize_max(mask)

        # Combine image with mask in alpha channel
        image_f.putalpha(mask_f)
        image_f.save("/content/ComfyUI/input/image.png")

        # Load the processed image into the ComfyUI pipeline
        image_ff, mask_ff = LoadImage.load_image("image.png")

        # Encode textual prompt and apply flux guidance
        cond = CLIPTextEncode.encode(text=prompt, clip=clip)[0]
        cond_f = FluxGuidance.append(conditioning=cond, guidance=30)[0]

        con_f_neg = CLIPTextEncode.encode(text="", clip=clip)[0]

        # Prepare inpaint conditioning
        cond_inpaint, cond_neg_inpaint, latent = InpaintModelConditioning.encode(
            positive=cond_f,
            negative=con_f_neg,
            vae=vae,
            pixels=image_ff,
            mask=mask_ff,
            noise_mask=False
        )

        noise = RandomNoise.get_noise(seed)[0]

        # Apply differential diffusion to the UNet model
        new_unet = DifferentialDiffusion.apply(model=unet)[0]

        # Perform sampling
        sample = KSampler.sample(
            model=new_unet,
            positive=cond_inpaint,
            negative=cond_neg_inpaint,
            latent_image=latent,
            seed=seed,
            steps=steps,
            cfg=1.0,
            sampler_name=sampler_name,
            scheduler=scheduler,
            denoise=1.0
        )[0]
        model_management.soft_empty_cache()

        # Decode from latent
        decoded = VAEDecode.decode(vae=vae, samples=sample)[0].detach()
        output_image = Image.fromarray(
            np.array(decoded * 255, dtype=np.uint8)[0]
        )

        return output_image


def on_generate_click(image, prompt, seed):
    """
    Callback function for the '이미지 생성하기' button in Gradio.
    Attempts to generate the image and returns the result and status.
    """
    if not prompt.strip():
        return None, "❌ 텍스트를 입력해주세요.", None
    if not image:
        return None, "❌ 이미지를 입력해주세요.", None
    try:
        generated = generate_image(image, prompt, seed)
        return generated, "✅ 성공적으로 이미지를 생성했습니다.", seed
    except Exception as e:
        return None, f"❌ 에러 발생: {str(e)}", None


def roll_seed():
    """
    Generates a random seed.
    """
    return random.randint(0, 2**32 - 1)


def create_interface():
    css = """
    footer {
        visibility: hidden;
    }
    """

    interface = gr.Interface(
        fn=on_generate_click,
        inputs=[
            gr.ImageEditor(
                label="🖼️ 이미지",
                type='filepath',
                sources="upload",
                brush=gr.Brush(
                    colors=["#FFFFFF"],
                    default_color="#FFFFFF",
                    color_mode="fixed"
                ),
                height="100%"
            ),
            gr.Textbox(
                label="📝 텍스트",
                placeholder="원하는 장면을 영어로 입력해주세요...",
                lines=3
            ),
            gr.Slider(0, 2**32 -1, 2024, label="🎲 시드", step=1),
        ],
        outputs=[
            gr.Image(label="🖼️ 생성한 이미지"),
            gr.Textbox(label="🔄 현재 상태", interactive=False),
            gr.Number(label="📌 사용된 시드", precision=0, interactive=False)
        ],
        title="🖼️ 플럭스 활용하기",
        theme='JohnSmith9982/small_and_pretty',
        css=css,
        allow_flagging="never"
    )
    return interface

In [ ]:
def main():
    # Load all models first
    load_models()

    # Create and launch the Gradio interface
    interface = create_interface()
    interface.launch()


if __name__ == "__main__":
    main()